<a href="https://colab.research.google.com/github/AnguillaJaponica/decision_analyzer/blob/master/chapter3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries
import numpy as np
import pandas as pd
# DataFrameの全角文字の出力をきれいにする。
pd.set_option('display.unicode.east_asian_width', True)

In [2]:
# 利得を計算する際のパラメータ
fixed_cost = 100
run_cost = 600
sales_price = 0.2

machine_ability = 5000
demand_boom = 10000
demand_slump = 5000

In [3]:
# 出荷される製品の個数
num_product_df = pd.DataFrame({
    '0台': [0, 0],
    '1台': [min([machine_ability, demand_boom]),
           min([machine_ability, demand_slump])],
    '2台': [min([machine_ability * 2, demand_boom]),
           min([machine_ability * 2, demand_slump])]
})
num_product_df.index = ['好況', '不況']
print(num_product_df)

      0台   1台    2台
好況    0  5000  10000
不況    0  5000   5000


In [4]:
# 売上行列
sales_df = num_product_df * sales_price
print(sales_df)

      0台     1台     2台
好況  0.0  1000.0  2000.0
不況  0.0  1000.0  1000.0


In [5]:
# 製造コスト
run_cost_df = pd.DataFrame({
    '0台': np.repeat(fixed_cost, 2),
    '1台': np.repeat(fixed_cost + run_cost, 2),
    '2台': np.repeat(fixed_cost + run_cost * 2, 2)
})
run_cost_df.index =  ['好況', '不況']
print(run_cost_df)

      0台  1台   2台
好況  100  700  1300
不況  100  700  1300


In [6]:
# 利得行列
paypff_df = sales_df - run_cost_df
print(paypff_df)

        0台    1台    2台
好況 -100.0  300.0  700.0
不況 -100.0  300.0 -300.0


In [7]:
# 利得行列を一発で作る関数

def calc_payoff_table(fixed_cost, run_cost, sales_price, machine_ability, demand_boom ,demand_slump):
  # 出荷される製品の個数
  num_product_df = pd.DataFrame({
      '0台': [0, 0],
      '1台': [min([machine_ability, demand_boom]),
            min([machine_ability, demand_slump])],
      '2台': [min([machine_ability * 2, demand_boom]),
            min([machine_ability * 2, demand_slump])]
  })
  sales_df = num_product_df * sales_price
  # 製造コスト
  run_cost_df = pd.DataFrame({
      '0台': np.repeat(fixed_cost, 2),
      '1台': np.repeat(fixed_cost + run_cost, 2),
      '2台': np.repeat(fixed_cost + run_cost * 2, 2)
  })
  # 利得行列
  paypff_df = sales_df - run_cost_df
  paypff_df.index =  ['好況', '不況']
  # 結果を返す
  return(paypff_df)


In [8]:
payoff = calc_payoff_table(fixed_cost = 100, run_cost = 600, sales_price = 0.2, machine_ability = 5000, demand_boom = 10000 ,demand_slump = 5000)

In [9]:
# 選択肢ごとの最大利得
payoff.max()

0台   -100.0
1台    300.0
2台    700.0
dtype: float64

In [22]:
payoff.max().max()

700.0

In [10]:
payoff.max() == payoff.max().max()

0台    False
1台    False
2台     True
dtype: bool

In [11]:
payoff.max()[payoff.max() == payoff.max().max()]

2台    700.0
dtype: float64

In [12]:
list(payoff.max()[payoff.max() == payoff.max().max()].index)

['2台']

In [17]:
# 最大値を取るインデックスを取得する。最大値が複数ある場合は全て取得する。
def argmax_list(series):
  return(list(series[series == series.max()].index))

# 最小値を取るインデックスを取得する。最小値が複数ある場合は全て取得する。
def argmin_list(series):
  return(list(series[series == series.min()].index))

In [16]:
print('Maximax: ', argmax_list(payoff.max()))

Maximax:  ['2台']


In [18]:
# 選択肢ごとの最小利得
payoff.min()

0台   -100.0
1台    300.0
2台   -300.0
dtype: float64

In [19]:
print('Maximin: ', argmax_list(payoff.min()))

Maximin:  ['1台']
